In [1]:
import pandas as pd
import numpy as np
import re
import json
import numpy as np
import timeit 
from datetime import datetime

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint

In [3]:
# Cufflinks wrapper on plotly
import cufflinks
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from plotly.offline import iplot
cufflinks.go_offline()

# Set global theme

import plotly.figure_factory as ff

import plotly.graph_objects as go
import plotly.express as px

In [4]:
import plotly.express as px
from shapely.geometry import Polygon, MultiPolygon
import geopandas as gpd
import matplotlib.pyplot as plt
import geojsoncontour

from scipy.interpolate import griddata
from numpy import linspace

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
def performance_summary(model, X_test, y_test ):
    
    y_hat=model.predict(X_test)
    
    df_summary=pd.DataFrame(y_hat, columns=["y_hat"])
    df_summary["y_true"]=y_test
    df_summary["abs_error"]=np.abs(df_summary.y_true-df_summary.y_hat)
    df_summary["error"]=df_summary.y_hat-df_summary.y_true
    df_summary["relative_error"]= df_summary["error"]/df_summary.y_true
    df_summary["relative_abs_error"]= df_summary["abs_error"]/df_summary.y_true
    
    share_within_5pct=(df_summary.query("relative_abs_error<0.05").shape[0]/df_summary.shape[0])*100
    
    print("{:.2f}% : Share of forecasts within 5% absolute error\n".format(share_within_5pct))
    print("{:.0f}   : Mean absolute error \n".format(df_summary.abs_error.mean()))
    print("{:.2f}% : Mean absolute percentage error\n".format(df_summary.relative_abs_error.mean()*100))



In [7]:
def DNN_plot_loss(history, starting_epoch,previous_val_loss):

        trace0=go.Scatter(
                y=history.history['loss'][starting_epoch:],
                x=history.epoch[starting_epoch:],
                mode='lines',
                marker=dict(
                color="blue",
                size=5,
                opacity=0.5
                ),
                name="Training Loss"
            )


        trace1=go.Scatter(
                y=history.history['val_loss'][starting_epoch:],
                x=history.epoch[starting_epoch:],
                mode='lines',
                marker=dict(
                color="red",
                size=5,
                opacity=0.5
                ),
                name="Validation Loss"
            )
        
        trace2=go.Scatter(
                y=list(np.ones([len(history.epoch[starting_epoch:])])*np.asarray(previous_val_loss).min()),
                x=history.epoch[starting_epoch:],
                mode='lines',
                marker=dict(
                color="grey",
                size=5,

                ),
                name="Lowest error from previous models"
            )

        data=[trace0, trace1,trace2]
        figure=go.Figure(
            data=data,
            layout=go.Layout(
                title="Learning curve",
                yaxis=dict(title="Loss",range=(900,1500)),
                xaxis=dict(title="Epoch",range=(starting_epoch,history.epoch[-1])),
                legend=dict(
                    x=1,
                    y=1,
                    traceorder="normal",
                    font=dict(
                        family="sans-serif",
                        size=12,
                        color="black"
                    ),
                bgcolor=None


            )))
        iplot(figure)

In [8]:
#Selecting a central city point to center all graphs around - Swietokrzyska Subway 
center_coors=52.235176, 21.008393

In [94]:
df = pd.read_csv(r"https://raw.githubusercontent.com/Jan-Majewski/Project_Portfolio/master/03_Real_Estate_pricing_in_Warsaw/top_features_data.csv")
df.shape

(24935, 46)

In [10]:
y=df.unit_price
X=df.drop(columns=["unit_price","lat_mod","lon_mod"])

## Spliting into train and test sets

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15)


In [12]:
test_idx=np.asarray(X_test.index)

In [13]:
X_test_coors=df[[ "unit_price","lat_mod","lon_mod"]].iloc[test_idx]
X_test_coors.reset_index(inplace=True,drop=True)

In [14]:
df.columns

Index(['build_year', 'building_floors_num', 'rooms_num',
       'Equipment_types_dishwasher', 'Equipment_types_fridge',
       'Equipment_types_furniture', 'Equipment_types_tv',
       'Equipment_types_washing_machine', 'Extras_types_air_conditioning',
       'Extras_types_balcony', 'Extras_types_garden', 'Extras_types_lift',
       'floor_num', 'east_bank', 'distance_driving', 'distance_transit',
       'time_driving', 'time_transit', 'restaurant_price_level',
       'restaurant_mean_rating', 'restaurant_mean_popularity',
       'restaurant_count', 'restaurant_ratings_count', 'district_Bemowo',
       'district_Bialoleka', 'district_Downtown', 'district_Subburbs',
       'district_Targowek', 'district_Wawer', 'district_Wola',
       'district_Zoliborz', 'market_primary', 'Building_material_brick',
       'Building_ownership_full_ownership', 'Building_type_apartment',
       'Building_type_block', 'Building_type_tenement',
       'Construction_status_ready_to_use', 'Construction_status

In [15]:
y_train.reset_index(drop=True,inplace=True)
y_test.reset_index(drop=True,inplace=True)

X_train.reset_index(drop=True,inplace=True)
X_test.reset_index(drop=True,inplace=True)

# Neural networks models

## Transforming and scaling data for NN

In [16]:
y_train=np.asarray(y_train).reshape(-1,1)
y_train.shape

(19948, 1)

In [17]:
y_train.mean()

11195.887256867856

In [18]:
y_test=np.asarray(y_test).reshape(-1,1)
y_test.shape

(4987, 1)

In [19]:
y_test.mean()

11128.766392620813

In [20]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [21]:
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [22]:
X_train.shape

(19948, 43)

In [23]:
X_test.shape

(4987, 43)

## Initial model

In [24]:
tf.keras.backend.clear_session()
tf.random.set_seed(60)

model=keras.models.Sequential([
    
    keras.layers.Dense(512, input_dim = X_train.shape[1], activation='relu'),  
    keras.layers.Dense(512, input_dim = X_train.shape[1], activation='relu'),  
    keras.layers.Dense(units=256,activation='relu'),  
    keras.layers.Dense(units=256,activation='relu'),    
    keras.layers.Dense(units=128,activation='relu'),
    keras.layers.Dense(units=1, activation="linear"),



],name="Initial_model",)


In [25]:
model.summary()

Model: "Initial_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               22528     
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_4 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 129       
Total params: 515,329
Trainable params: 515,329
Non-trainable params: 0
_______________________________________________

In [26]:
optimizer = keras.optimizers.Adam()



model.compile(optimizer=optimizer, warm_start=False, 
            loss='mean_absolute_error')


history = model.fit(X_train, y_train,
                    epochs=200, batch_size=1024,
                    validation_data=(X_test, y_test), 
                    verbose=1)

Train on 19948 samples, validate on 4987 samples
Epoch 1/200
19948/19948 [==============================] - 3s 153us/sample - loss: 11118.3712 - val_loss: 10586.6312
Epoch 2/200
19948/19948 [==============================] - 0s 24us/sample - loss: 7066.7532 - val_loss: 3763.7940
Epoch 3/200
19948/19948 [==============================] - ETA: 0s - loss: 2550.48 - 0s 18us/sample - loss: 2464.6214 - val_loss: 1684.2396
Epoch 4/200
19948/19948 [==============================] - 1s 33us/sample - loss: 1628.4329 - val_loss: 1496.9288
Epoch 5/200
19948/19948 [==============================] - 0s 18us/sample - loss: 1463.1655 - val_loss: 1407.6184
Epoch 6/200
19948/19948 [==============================] - 0s 9us/sample - loss: 1387.9317 - val_loss: 1362.7423
Epoch 7/200
19948/19948 [==============================] - 0s 11us/sample - loss: 1347.6675 - val_loss: 1336.8411
Epoch 8/200
19948/19948 [==============================] - 0s 9us/sample - loss: 1319.9526 - val_loss: 1317.9871
Epoch 9/200


19948/19948 [==============================] - 0s 8us/sample - loss: 1041.5853 - val_loss: 1177.2874
Epoch 73/200
19948/19948 [==============================] - 0s 9us/sample - loss: 1038.3217 - val_loss: 1182.3154
Epoch 74/200
19948/19948 [==============================] - 0s 8us/sample - loss: 1035.1507 - val_loss: 1193.8523
Epoch 75/200
19948/19948 [==============================] - 0s 8us/sample - loss: 1040.1218 - val_loss: 1183.0535
Epoch 76/200
19948/19948 [==============================] - 0s 8us/sample - loss: 1032.4921 - val_loss: 1186.0599
Epoch 77/200
19948/19948 [==============================] - 0s 8us/sample - loss: 1021.8529 - val_loss: 1172.1018
Epoch 78/200
19948/19948 [==============================] - 0s 8us/sample - loss: 1012.7016 - val_loss: 1173.8347
Epoch 79/200
19948/19948 [==============================] - 0s 8us/sample - loss: 1014.0081 - val_loss: 1191.0554
Epoch 80/200
19948/19948 [==============================] - 0s 9us/sample - loss: 1014.2976 - val_los

Epoch 144/200
19948/19948 [==============================] - 0s 9us/sample - loss: 799.8053 - val_loss: 1178.5600
Epoch 145/200
19948/19948 [==============================] - 0s 10us/sample - loss: 803.7892 - val_loss: 1181.5303
Epoch 146/200
19948/19948 [==============================] - 0s 10us/sample - loss: 785.4978 - val_loss: 1191.8789
Epoch 147/200
19948/19948 [==============================] - 0s 9us/sample - loss: 780.3535 - val_loss: 1180.4938
Epoch 148/200
19948/19948 [==============================] - 0s 10us/sample - loss: 792.4177 - val_loss: 1195.4573
Epoch 149/200
19948/19948 [==============================] - 0s 11us/sample - loss: 813.9468 - val_loss: 1200.4485
Epoch 150/200
19948/19948 [==============================] - 0s 15us/sample - loss: 800.3382 - val_loss: 1185.5327
Epoch 151/200
19948/19948 [==============================] - 0s 12us/sample - loss: 789.7976 - val_loss: 1180.2982
Epoch 152/200
19948/19948 [==============================] - 0s 9us/sample - loss:

In [27]:
DNN_plot_loss(history,starting_epoch=10, previous_val_loss=[0])

In [28]:
previous_val_loss=[history.history["val_loss"][-1]]
performance_summary(model, X_test, y_test )

39.46% : Share of forecasts within 5% absolute error

1191   : Mean absolute error 

10.21% : Mean absolute percentage error



## Adding Drop-out

In [29]:
tf.keras.backend.clear_session()
tf.random.set_seed(60)

model=keras.models.Sequential([
    
    keras.layers.Dense(512, input_dim = X_train.shape[1], activation='relu'),  
    keras.layers.Dropout(0.3),
    
    keras.layers.Dense(512, activation='relu'),  
    keras.layers.Dropout(0.3),

    keras.layers.Dense(units=256,activation='relu'), 
    keras.layers.Dropout(0.2),
    
    keras.layers.Dense(units=256,activation='relu'), 
    keras.layers.Dropout(0.2),
    
    keras.layers.Dense(units=128,activation='relu'),
    keras.layers.Dense(units=1, activation="linear"),



],name="Dropout",)


In [30]:
model.summary()

Model: "Dropout"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               22528     
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               6579

In [31]:
optimizer = keras.optimizers.Adam()



model.compile(optimizer=optimizer, warm_start=False, 
            loss='mean_absolute_error')


history = model.fit(X_train, y_train,
                    epochs=200, batch_size=1024,
                    validation_data=(X_test, y_test), 
                    verbose=1)

Train on 19948 samples, validate on 4987 samples
Epoch 1/200
19948/19948 [==============================] - 1s 46us/sample - loss: 11104.9790 - val_loss: 10453.7489
Epoch 2/200
19948/19948 [==============================] - 0s 9us/sample - loss: 6566.8866 - val_loss: 2844.8541
Epoch 3/200
19948/19948 [==============================] - 0s 10us/sample - loss: 2426.2825 - val_loss: 1743.4334
Epoch 4/200
19948/19948 [==============================] - 0s 11us/sample - loss: 1758.0013 - val_loss: 1499.9645
Epoch 5/200
19948/19948 [==============================] - 0s 10us/sample - loss: 1648.7797 - val_loss: 1414.9995
Epoch 6/200
19948/19948 [==============================] - 0s 9us/sample - loss: 1603.1394 - val_loss: 1372.3611
Epoch 7/200
19948/19948 [==============================] - 0s 9us/sample - loss: 1572.5695 - val_loss: 1345.1777
Epoch 8/200
19948/19948 [==============================] - 0s 21us/sample - loss: 1534.9278 - val_loss: 1323.9774
Epoch 9/200
19948/19948 [===============

19948/19948 [==============================] - 0s 14us/sample - loss: 1344.8349 - val_loss: 1201.0028
Epoch 73/200
19948/19948 [==============================] - 0s 17us/sample - loss: 1356.0631 - val_loss: 1210.5179
Epoch 74/200
19948/19948 [==============================] - 0s 13us/sample - loss: 1358.5342 - val_loss: 1191.1674
Epoch 75/200
19948/19948 [==============================] - 0s 9us/sample - loss: 1356.5611 - val_loss: 1194.8789
Epoch 76/200
19948/19948 [==============================] - 0s 17us/sample - loss: 1369.2569 - val_loss: 1205.6773
Epoch 77/200
19948/19948 [==============================] - 0s 16us/sample - loss: 1351.4459 - val_loss: 1221.7578
Epoch 78/200
19948/19948 [==============================] - 0s 14us/sample - loss: 1357.5692 - val_loss: 1198.2149
Epoch 79/200
19948/19948 [==============================] - 0s 18us/sample - loss: 1346.9332 - val_loss: 1200.3566
Epoch 80/200
19948/19948 [==============================] - 0s 11us/sample - loss: 1359.4658 -

19948/19948 [==============================] - 0s 9us/sample - loss: 1324.2511 - val_loss: 1164.2917
Epoch 144/200
19948/19948 [==============================] - 0s 9us/sample - loss: 1296.6610 - val_loss: 1158.4953
Epoch 145/200
19948/19948 [==============================] - 0s 11us/sample - loss: 1301.7553 - val_loss: 1174.9954
Epoch 146/200
19948/19948 [==============================] - 0s 10us/sample - loss: 1317.4221 - val_loss: 1191.8962
Epoch 147/200
19948/19948 [==============================] - 0s 10us/sample - loss: 1298.6759 - val_loss: 1164.5919
Epoch 148/200
19948/19948 [==============================] - 0s 9us/sample - loss: 1306.2641 - val_loss: 1184.8389
Epoch 149/200
19948/19948 [==============================] - 0s 10us/sample - loss: 1295.3704 - val_loss: 1168.0473
Epoch 150/200
19948/19948 [==============================] - 0s 8us/sample - loss: 1305.6053 - val_loss: 1166.7980
Epoch 151/200
19948/19948 [==============================] - 0s 10us/sample - loss: 1291.1

In [32]:
DNN_plot_loss(history,starting_epoch=10, previous_val_loss=previous_val_loss)

In [33]:
previous_val_loss.append(history.history["val_loss"][-1])
performance_summary(model, X_test, y_test )

38.24% : Share of forecasts within 5% absolute error

1157   : Mean absolute error 

9.87% : Mean absolute percentage error



## Adding batch normalization

In [34]:
tf.keras.backend.clear_session()
tf.random.set_seed(60)

model=keras.models.Sequential([
    
    keras.layers.Dense(512, input_dim = X_train.shape[1], activation='relu'), 
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    
    keras.layers.Dense(512, activation='relu'),  
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),

    keras.layers.Dense(units=256,activation='relu'), 
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.2),
    
    keras.layers.Dense(units=256,activation='relu'), 
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.2),
    
    keras.layers.Dense(units=128,activation='relu'),
    keras.layers.Dense(units=1, activation="linear"),



],name="Batchnorm",)


In [35]:
model.summary()

Model: "Batchnorm"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               22528     
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               13

In [36]:
optimizer = keras.optimizers.Adam()



model.compile(optimizer=optimizer, warm_start=False, 
            loss='mean_absolute_error')


history = model.fit(X_train, y_train,
                    epochs=200, batch_size=1024,
                    validation_data=(X_test, y_test), 
                    verbose=1)

Train on 19948 samples, validate on 4987 samples
Epoch 1/200
19948/19948 [==============================] - 2s 120us/sample - loss: 11193.2647 - val_loss: 11118.9971
Epoch 2/200
19948/19948 [==============================] - 0s 16us/sample - loss: 11176.2128 - val_loss: 11081.1694
Epoch 3/200
19948/19948 [==============================] - 0s 17us/sample - loss: 11140.8975 - val_loss: 11036.2493
Epoch 4/200
19948/19948 [==============================] - 0s 18us/sample - loss: 11087.0119 - val_loss: 10963.5245
Epoch 5/200
19948/19948 [==============================] - 0s 15us/sample - loss: 11010.1089 - val_loss: 10843.1981
Epoch 6/200
19948/19948 [==============================] - 0s 15us/sample - loss: 10903.1804 - val_loss: 10675.0540
Epoch 7/200
19948/19948 [==============================] - 0s 15us/sample - loss: 10760.0586 - val_loss: 10465.3191
Epoch 8/200
19948/19948 [==============================] - 0s 18us/sample - loss: 10573.3401 - val_loss: 10204.3620
Epoch 9/200
19948/1994

19948/19948 [==============================] - 0s 15us/sample - loss: 1188.5822 - val_loss: 1129.8644
Epoch 72/200
19948/19948 [==============================] - 0s 16us/sample - loss: 1188.0753 - val_loss: 1122.6812
Epoch 73/200
19948/19948 [==============================] - 0s 15us/sample - loss: 1166.5342 - val_loss: 1128.6818
Epoch 74/200
19948/19948 [==============================] - 0s 15us/sample - loss: 1201.1976 - val_loss: 1125.9392
Epoch 75/200
19948/19948 [==============================] - 0s 15us/sample - loss: 1171.3940 - val_loss: 1140.2794
Epoch 76/200
19948/19948 [==============================] - 0s 15us/sample - loss: 1151.4845 - val_loss: 1127.8106
Epoch 77/200
19948/19948 [==============================] - 0s 15us/sample - loss: 1160.4606 - val_loss: 1132.2761
Epoch 78/200
19948/19948 [==============================] - 0s 16us/sample - loss: 1169.6768 - val_loss: 1125.9864
Epoch 79/200
19948/19948 [==============================] - 0s 15us/sample - loss: 1159.0885 

Epoch 142/200
19948/19948 [==============================] - 0s 16us/sample - loss: 1027.0748 - val_loss: 1092.4569
Epoch 143/200
19948/19948 [==============================] - 0s 16us/sample - loss: 1005.4187 - val_loss: 1105.6217
Epoch 144/200
19948/19948 [==============================] - 0s 17us/sample - loss: 1015.4161 - val_loss: 1090.5424
Epoch 145/200
19948/19948 [==============================] - 0s 15us/sample - loss: 1029.2523 - val_loss: 1086.4205
Epoch 146/200
19948/19948 [==============================] - 0s 15us/sample - loss: 1003.4030 - val_loss: 1091.2997
Epoch 147/200
19948/19948 [==============================] - 0s 21us/sample - loss: 1019.5471 - val_loss: 1090.0725
Epoch 148/200
19948/19948 [==============================] - 0s 15us/sample - loss: 1005.0308 - val_loss: 1095.7604
Epoch 149/200
19948/19948 [==============================] - 0s 14us/sample - loss: 1077.5319 - val_loss: 1088.8389
Epoch 150/200
19948/19948 [==============================] - 0s 14us/sam

In [37]:
previous_val_loss[-1]

1156.7920102030905

In [38]:
DNN_plot_loss(history,starting_epoch=10, previous_val_loss=previous_val_loss)

In [39]:
previous_val_loss.append(history.history["val_loss"][-1])
performance_summary(model, X_test, y_test )

42.81% : Share of forecasts within 5% absolute error

1082   : Mean absolute error 

9.29% : Mean absolute percentage error



## Adding Leaky Relu

In [40]:
tf.keras.backend.clear_session()
tf.random.set_seed(60)

model=keras.models.Sequential([
    
    keras.layers.Dense(512, input_dim = X_train.shape[1]), 
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    
    keras.layers.Dense(512),  
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),

    keras.layers.Dense(units=256), 
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.2),
    
    keras.layers.Dense(units=256), 
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.2),
    
    keras.layers.Dense(units=128),
    keras.layers.LeakyReLU(), 
    keras.layers.Dense(units=1, activation="linear"),



],name="LeakyRELU",)


In [41]:
model.summary()

Model: "LeakyRELU"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               22528     
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 512)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               20

In [42]:
optimizer = keras.optimizers.Adam()



model.compile(optimizer=optimizer, warm_start=False, 
            loss='mean_absolute_error')


history = model.fit(X_train, y_train,
                    epochs=300, batch_size=1024,
                    validation_data=(X_test, y_test), 
                    verbose=1)

Train on 19948 samples, validate on 4987 samples
Epoch 1/300
19948/19948 [==============================] - 2s 115us/sample - loss: 11193.0127 - val_loss: 11115.5698
Epoch 2/300
19948/19948 [==============================] - 0s 15us/sample - loss: 11179.0485 - val_loss: 11076.8315
Epoch 3/300
19948/19948 [==============================] - 0s 14us/sample - loss: 11154.1232 - val_loss: 11051.6703
Epoch 4/300
19948/19948 [==============================] - 0s 17us/sample - loss: 11116.2467 - val_loss: 11000.4914
Epoch 5/300
19948/19948 [==============================] - 0s 15us/sample - loss: 11061.6512 - val_loss: 10915.3436
Epoch 6/300
19948/19948 [==============================] - 0s 14us/sample - loss: 10985.3104 - val_loss: 10811.8844
Epoch 7/300
19948/19948 [==============================] - 0s 15us/sample - loss: 10882.4020 - val_loss: 10668.3015
Epoch 8/300
19948/19948 [==============================] - 0s 16us/sample - loss: 10748.2833 - val_loss: 10517.6703
Epoch 9/300
19948/1994

19948/19948 [==============================] - 0s 13us/sample - loss: 1304.7439 - val_loss: 1185.1122
Epoch 72/300
19948/19948 [==============================] - 0s 13us/sample - loss: 1297.0761 - val_loss: 1199.0675
Epoch 73/300
19948/19948 [==============================] - 0s 15us/sample - loss: 1293.2489 - val_loss: 1173.3242
Epoch 74/300
19948/19948 [==============================] - 0s 13us/sample - loss: 1296.2010 - val_loss: 1185.4635
Epoch 75/300
19948/19948 [==============================] - 0s 14us/sample - loss: 1295.6318 - val_loss: 1177.3871
Epoch 76/300
19948/19948 [==============================] - 0s 13us/sample - loss: 1289.5140 - val_loss: 1177.8125
Epoch 77/300
19948/19948 [==============================] - 0s 15us/sample - loss: 1282.3929 - val_loss: 1171.6203
Epoch 78/300
19948/19948 [==============================] - 0s 13us/sample - loss: 1300.3427 - val_loss: 1192.2241
Epoch 79/300
19948/19948 [==============================] - 0s 15us/sample - loss: 1283.6681 

Epoch 142/300
19948/19948 [==============================] - 0s 14us/sample - loss: 1222.5796 - val_loss: 1154.9052
Epoch 143/300
19948/19948 [==============================] - 0s 13us/sample - loss: 1224.0068 - val_loss: 1167.8904
Epoch 144/300
19948/19948 [==============================] - 0s 17us/sample - loss: 1224.4309 - val_loss: 1170.5528
Epoch 145/300
19948/19948 [==============================] - 0s 14us/sample - loss: 1214.5126 - val_loss: 1157.6741
Epoch 146/300
19948/19948 [==============================] - 0s 13us/sample - loss: 1229.9901 - val_loss: 1149.1673
Epoch 147/300
19948/19948 [==============================] - 0s 13us/sample - loss: 1236.8966 - val_loss: 1199.9494
Epoch 148/300
19948/19948 [==============================] - 0s 14us/sample - loss: 1227.0992 - val_loss: 1147.4083
Epoch 149/300
19948/19948 [==============================] - 0s 13us/sample - loss: 1225.3673 - val_loss: 1143.2794
Epoch 150/300
19948/19948 [==============================] - 0s 13us/sam

19948/19948 [==============================] - 0s 17us/sample - loss: 1184.2458 - val_loss: 1134.3451
Epoch 213/300
19948/19948 [==============================] - 0s 19us/sample - loss: 1180.3436 - val_loss: 1132.3320
Epoch 214/300
19948/19948 [==============================] - 0s 14us/sample - loss: 1192.5994 - val_loss: 1144.7656
Epoch 215/300
19948/19948 [==============================] - 0s 17us/sample - loss: 1180.0101 - val_loss: 1128.9191
Epoch 216/300
19948/19948 [==============================] - 0s 24us/sample - loss: 1176.9713 - val_loss: 1134.1195
Epoch 217/300
19948/19948 [==============================] - 0s 16us/sample - loss: 1186.1881 - val_loss: 1153.6358
Epoch 218/300
19948/19948 [==============================] - 0s 17us/sample - loss: 1187.4133 - val_loss: 1133.1147
Epoch 219/300
19948/19948 [==============================] - 0s 16us/sample - loss: 1182.1510 - val_loss: 1127.6236
Epoch 220/300
19948/19948 [==============================] - 0s 16us/sample - loss: 11

Epoch 283/300
19948/19948 [==============================] - 0s 20us/sample - loss: 1150.7890 - val_loss: 1119.1888
Epoch 284/300
19948/19948 [==============================] - 0s 14us/sample - loss: 1148.6266 - val_loss: 1125.2547
Epoch 285/300
19948/19948 [==============================] - 0s 14us/sample - loss: 1149.5249 - val_loss: 1119.9348
Epoch 286/300
19948/19948 [==============================] - 0s 13us/sample - loss: 1148.8255 - val_loss: 1128.7576
Epoch 287/300
19948/19948 [==============================] - 0s 13us/sample - loss: 1142.0614 - val_loss: 1120.1838
Epoch 288/300
19948/19948 [==============================] - 0s 13us/sample - loss: 1153.1345 - val_loss: 1127.9053
Epoch 289/300
19948/19948 [==============================] - 0s 15us/sample - loss: 1155.0581 - val_loss: 1124.1673
Epoch 290/300
19948/19948 [==============================] - 0s 13us/sample - loss: 1144.8657 - val_loss: 1133.0040
Epoch 291/300
19948/19948 [==============================] - 0s 13us/sam

In [43]:
DNN_plot_loss(history,starting_epoch=10, previous_val_loss=previous_val_loss)

In [44]:
previous_val_loss.append(history.history["val_loss"][-1])
performance_summary(model, X_test, y_test )

40.32% : Share of forecasts within 5% absolute error

1120   : Mean absolute error 

9.53% : Mean absolute percentage error



## Creating a larger network

In [45]:
tf.keras.backend.clear_session()
tf.random.set_seed(60)

model=keras.models.Sequential([
    
    keras.layers.Dense(1024, input_dim = X_train.shape[1]), 
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.5),
    
    keras.layers.Dense(512),  
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.4),

    keras.layers.Dense(512),  
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    
    keras.layers.Dense(units=256), 
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.2),
    
    keras.layers.Dense(units=256), 
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.1),

    
    keras.layers.Dense(units=128),
    keras.layers.LeakyReLU(), 
    keras.layers.Dense(units=1, activation="linear"),



],name="Larger_network",)


In [46]:
model.summary()

Model: "Larger_network"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1024)              45056     
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 1024)              0         
_________________________________________________________________
batch_normalization (BatchNo (None, 1024)              4096      
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)            

In [47]:
optimizer = keras.optimizers.Adam()



model.compile(optimizer=optimizer, warm_start=False, 
            loss='mean_absolute_error')


history = model.fit(X_train, y_train,
                    epochs=300, batch_size=1024,
                    validation_data=(X_test, y_test), 
                    verbose=1)

Train on 19948 samples, validate on 4987 samples
Epoch 1/300
19948/19948 [==============================] - 3s 136us/sample - loss: 11192.2855 - val_loss: 11104.1744
Epoch 2/300
19948/19948 [==============================] - 0s 19us/sample - loss: 11179.7319 - val_loss: 11044.3999
Epoch 3/300
19948/19948 [==============================] - 0s 18us/sample - loss: 11154.4925 - val_loss: 11009.9138
Epoch 4/300
19948/19948 [==============================] - 0s 20us/sample - loss: 11111.3551 - val_loss: 10935.6949
Epoch 5/300
19948/19948 [==============================] - 0s 18us/sample - loss: 11050.1698 - val_loss: 10821.2224
Epoch 6/300
19948/19948 [==============================] - 0s 16us/sample - loss: 10966.7383 - val_loss: 10698.1283
Epoch 7/300
19948/19948 [==============================] - 0s 17us/sample - loss: 10856.3857 - val_loss: 10556.7407
Epoch 8/300
19948/19948 [==============================] - 0s 17us/sample - loss: 10712.9032 - val_loss: 10378.4995
Epoch 9/300
19948/1994

19948/19948 [==============================] - 0s 19us/sample - loss: 1282.6997 - val_loss: 1188.6751
Epoch 72/300
19948/19948 [==============================] - 0s 25us/sample - loss: 1279.2504 - val_loss: 1199.4892
Epoch 73/300
19948/19948 [==============================] - 0s 22us/sample - loss: 1271.5428 - val_loss: 1183.5082
Epoch 74/300
19948/19948 [==============================] - 0s 20us/sample - loss: 1261.7858 - val_loss: 1194.8201
Epoch 75/300
19948/19948 [==============================] - 0s 18us/sample - loss: 1267.9999 - val_loss: 1184.3130
Epoch 76/300
19948/19948 [==============================] - 0s 18us/sample - loss: 1259.7683 - val_loss: 1190.8611
Epoch 77/300
19948/19948 [==============================] - 0s 22us/sample - loss: 1265.1761 - val_loss: 1190.2496
Epoch 78/300
19948/19948 [==============================] - 0s 18us/sample - loss: 1270.1030 - val_loss: 1182.9975
Epoch 79/300
19948/19948 [==============================] - 0s 18us/sample - loss: 1257.8939 

Epoch 142/300
19948/19948 [==============================] - 0s 19us/sample - loss: 1201.2993 - val_loss: 1151.4612
Epoch 143/300
19948/19948 [==============================] - 0s 18us/sample - loss: 1192.4619 - val_loss: 1141.1378
Epoch 144/300
19948/19948 [==============================] - 0s 19us/sample - loss: 1201.2617 - val_loss: 1144.8733
Epoch 145/300
19948/19948 [==============================] - 0s 21us/sample - loss: 1191.4896 - val_loss: 1148.6613
Epoch 146/300
19948/19948 [==============================] - 0s 18us/sample - loss: 1197.8543 - val_loss: 1143.1152
Epoch 147/300
19948/19948 [==============================] - 0s 17us/sample - loss: 1200.8862 - val_loss: 1139.4277
Epoch 148/300
19948/19948 [==============================] - 0s 17us/sample - loss: 1195.6609 - val_loss: 1138.2745
Epoch 149/300
19948/19948 [==============================] - 0s 16us/sample - loss: 1195.0361 - val_loss: 1138.6611
Epoch 150/300
19948/19948 [==============================] - 0s 16us/sam

19948/19948 [==============================] - 0s 18us/sample - loss: 1162.0059 - val_loss: 1126.3242
Epoch 213/300
19948/19948 [==============================] - 0s 17us/sample - loss: 1151.5496 - val_loss: 1123.0265
Epoch 214/300
19948/19948 [==============================] - 0s 18us/sample - loss: 1161.5447 - val_loss: 1120.1518
Epoch 215/300
19948/19948 [==============================] - 0s 16us/sample - loss: 1148.8807 - val_loss: 1118.6170
Epoch 216/300
19948/19948 [==============================] - 0s 17us/sample - loss: 1155.8507 - val_loss: 1117.2882
Epoch 217/300
19948/19948 [==============================] - 0s 16us/sample - loss: 1153.5685 - val_loss: 1117.5153
Epoch 218/300
19948/19948 [==============================] - 0s 17us/sample - loss: 1160.9653 - val_loss: 1112.6557
Epoch 219/300
19948/19948 [==============================] - 0s 17us/sample - loss: 1143.9704 - val_loss: 1115.1119
Epoch 220/300
19948/19948 [==============================] - 0s 16us/sample - loss: 11

Epoch 283/300
19948/19948 [==============================] - 0s 16us/sample - loss: 1127.1820 - val_loss: 1105.6695
Epoch 284/300
19948/19948 [==============================] - 0s 17us/sample - loss: 1129.3647 - val_loss: 1109.6902
Epoch 285/300
19948/19948 [==============================] - 0s 16us/sample - loss: 1115.6847 - val_loss: 1101.0518
Epoch 286/300
19948/19948 [==============================] - 0s 16us/sample - loss: 1116.4707 - val_loss: 1107.8980
Epoch 287/300
19948/19948 [==============================] - 0s 17us/sample - loss: 1122.0188 - val_loss: 1101.3935
Epoch 288/300
19948/19948 [==============================] - 0s 17us/sample - loss: 1123.7177 - val_loss: 1100.3120
Epoch 289/300
19948/19948 [==============================] - 0s 17us/sample - loss: 1124.8317 - val_loss: 1103.8309
Epoch 290/300
19948/19948 [==============================] - 0s 17us/sample - loss: 1128.2318 - val_loss: 1098.2733
Epoch 291/300
19948/19948 [==============================] - 0s 17us/sam

In [48]:
DNN_plot_loss(history,starting_epoch=10, previous_val_loss=previous_val_loss)

In [49]:
previous_val_loss.append(history.history["val_loss"][-1])
performance_summary(model, X_test, y_test )

40.73% : Share of forecasts within 5% absolute error

1098   : Mean absolute error 

9.41% : Mean absolute percentage error



## Adjusting learning rate decay

In [95]:
tf.keras.backend.clear_session()
tf.random.set_seed(60)

model=keras.models.Sequential([
    
    keras.layers.Dense(1024, input_dim = X_train.shape[1]), 
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.4),
    
    keras.layers.Dense(512),  
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),

    keras.layers.Dense(512),  
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    
    keras.layers.Dense(units=256), 
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.2),
    
    keras.layers.Dense(units=256), 
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.1),

    
    keras.layers.Dense(units=128),
    keras.layers.LeakyReLU(), 
    keras.layers.Dropout(0.05),
    keras.layers.Dense(units=1, activation="linear"),



],name="Learning_rate_decay",)


In [96]:
learning_rate = 0.005
decay = 1e-3
n_epochs=300
n_steps_per_epoch = len(X_train) // 1024
epochs = np.arange(n_epochs)
lrs = learning_rate / (1 + decay * epochs * n_steps_per_epoch)

In [97]:


trace0=go.Scatter(
            y=lrs,
            x=epochs,
            mode='lines',
            marker=dict(
            color="red",
            size=5,
            opacity=0.5
            )
    )
        

data=[trace0]
figure=go.Figure(
            data=data,
            layout=go.Layout(
                title="Learning curve",
                yaxis=dict(title="Learning rate"),
                xaxis=dict(title="Epoch"),
                legend=dict(
                    x=1,
                    y=1,
                    traceorder="normal",
                    font=dict(
                        family="sans-serif",
                        size=12,
                        color="black"
                    ),
                bgcolor=None


            )))
iplot(figure)

In [98]:
optimizer = keras.optimizers.Adam(lr=0.005, decay=1e-3)



model.compile(optimizer=optimizer, warm_start=False, 
            loss='mean_absolute_error')


history = model.fit(X_train, y_train,
                    epochs=400, batch_size=1024,
                    validation_data=(X_test, y_test), 
                    verbose=1)

Train on 19948 samples, validate on 4987 samples
Epoch 1/400
19948/19948 [==============================] - 3s 140us/sample - loss: 11171.9912 - val_loss: 10171.6175
Epoch 2/400
19948/19948 [==============================] - 1s 39us/sample - loss: 10981.1403 - val_loss: 9519.8079
Epoch 3/400
19948/19948 [==============================] - 1s 26us/sample - loss: 10367.3199 - val_loss: 7049.3895
Epoch 4/400
19948/19948 [==============================] - 1s 31us/sample - loss: 8996.3459 - val_loss: 5648.5767
Epoch 5/400
19948/19948 [==============================] - 1s 34us/sample - loss: 6523.2488 - val_loss: 6956.5481
Epoch 6/400
19948/19948 [==============================] - 1s 32us/sample - loss: 3523.3090 - val_loss: 7441.0197
Epoch 7/400
19948/19948 [==============================] - 0s 24us/sample - loss: 1772.8196 - val_loss: 12057.2688
Epoch 8/400
19948/19948 [==============================] - 1s 26us/sample - loss: 1617.2185 - val_loss: 10632.2914
Epoch 9/400
19948/19948 [=======

Epoch 72/400
19948/19948 [==============================] - 0s 24us/sample - loss: 1244.8074 - val_loss: 1161.9507
Epoch 73/400
19948/19948 [==============================] - 0s 21us/sample - loss: 1230.9441 - val_loss: 1162.1774
Epoch 74/400
19948/19948 [==============================] - 0s 25us/sample - loss: 1233.3487 - val_loss: 1158.4019
Epoch 75/400
19948/19948 [==============================] - 0s 21us/sample - loss: 1226.2658 - val_loss: 1174.6435
Epoch 76/400
19948/19948 [==============================] - 0s 19us/sample - loss: 1234.9970 - val_loss: 1171.7485
Epoch 77/400
19948/19948 [==============================] - 0s 20us/sample - loss: 1227.0185 - val_loss: 1179.9511
Epoch 78/400
19948/19948 [==============================] - 0s 22us/sample - loss: 1227.5357 - val_loss: 1160.8970
Epoch 79/400
19948/19948 [==============================] - 0s 20us/sample - loss: 1221.1107 - val_loss: 1172.6220
Epoch 80/400
19948/19948 [==============================] - 0s 17us/sample - los

Epoch 143/400
19948/19948 [==============================] - 0s 25us/sample - loss: 1167.5881 - val_loss: 1126.2949
Epoch 144/400
19948/19948 [==============================] - 0s 19us/sample - loss: 1163.2005 - val_loss: 1133.9120
Epoch 145/400
19948/19948 [==============================] - 0s 18us/sample - loss: 1164.4124 - val_loss: 1127.9940
Epoch 146/400
19948/19948 [==============================] - 0s 18us/sample - loss: 1172.9892 - val_loss: 1126.6358
Epoch 147/400
19948/19948 [==============================] - 0s 18us/sample - loss: 1168.2314 - val_loss: 1130.8351
Epoch 148/400
19948/19948 [==============================] - 0s 19us/sample - loss: 1155.9621 - val_loss: 1130.8051
Epoch 149/400
19948/19948 [==============================] - 0s 17us/sample - loss: 1165.6345 - val_loss: 1128.1870
Epoch 150/400
19948/19948 [==============================] - 0s 18us/sample - loss: 1166.2103 - val_loss: 1124.9474
Epoch 151/400
19948/19948 [==============================] - 0s 19us/sam

19948/19948 [==============================] - 0s 18us/sample - loss: 1131.9956 - val_loss: 1109.7509
Epoch 214/400
19948/19948 [==============================] - 0s 17us/sample - loss: 1114.7326 - val_loss: 1106.2844
Epoch 215/400
19948/19948 [==============================] - 0s 18us/sample - loss: 1121.2656 - val_loss: 1105.0983
Epoch 216/400
19948/19948 [==============================] - 0s 19us/sample - loss: 1116.7079 - val_loss: 1104.8909
Epoch 217/400
19948/19948 [==============================] - 0s 17us/sample - loss: 1133.8237 - val_loss: 1109.6679
Epoch 218/400
19948/19948 [==============================] - 0s 18us/sample - loss: 1125.0377 - val_loss: 1111.6402
Epoch 219/400
19948/19948 [==============================] - 0s 17us/sample - loss: 1132.4009 - val_loss: 1111.0711
Epoch 220/400
19948/19948 [==============================] - 0s 17us/sample - loss: 1119.8802 - val_loss: 1105.2066
Epoch 221/400
19948/19948 [==============================] - 0s 19us/sample - loss: 11

19948/19948 [==============================] - 0s 17us/sample - loss: 1101.7443 - val_loss: 1100.1086
Epoch 284/400
19948/19948 [==============================] - 0s 18us/sample - loss: 1110.6791 - val_loss: 1103.5713
Epoch 285/400
19948/19948 [==============================] - 0s 23us/sample - loss: 1096.0764 - val_loss: 1099.6288
Epoch 286/400
19948/19948 [==============================] - 0s 18us/sample - loss: 1096.3791 - val_loss: 1103.8458
Epoch 287/400
19948/19948 [==============================] - 0s 19us/sample - loss: 1099.9575 - val_loss: 1099.4184
Epoch 288/400
19948/19948 [==============================] - 0s 20us/sample - loss: 1103.2459 - val_loss: 1106.6235
Epoch 289/400
19948/19948 [==============================] - 0s 23us/sample - loss: 1101.4990 - val_loss: 1102.6933
Epoch 290/400
19948/19948 [==============================] - 0s 20us/sample - loss: 1102.7079 - val_loss: 1097.8390
Epoch 291/400
19948/19948 [==============================] - 1s 26us/sample - loss: 10

Epoch 354/400
19948/19948 [==============================] - 0s 17us/sample - loss: 1083.5307 - val_loss: 1100.5034
Epoch 355/400
19948/19948 [==============================] - 0s 19us/sample - loss: 1084.6343 - val_loss: 1095.3567
Epoch 356/400
19948/19948 [==============================] - 0s 22us/sample - loss: 1086.1410 - val_loss: 1095.6316
Epoch 357/400
19948/19948 [==============================] - 0s 20us/sample - loss: 1086.2421 - val_loss: 1095.8183
Epoch 358/400
19948/19948 [==============================] - 0s 21us/sample - loss: 1071.6851 - val_loss: 1096.9199
Epoch 359/400
19948/19948 [==============================] - 0s 18us/sample - loss: 1081.3995 - val_loss: 1091.7221
Epoch 360/400
19948/19948 [==============================] - 0s 21us/sample - loss: 1074.2338 - val_loss: 1093.8421
Epoch 361/400
19948/19948 [==============================] - 0s 21us/sample - loss: 1085.1035 - val_loss: 1097.1383
Epoch 362/400
19948/19948 [==============================] - 0s 24us/sam

In [99]:
DNN_plot_loss(history,starting_epoch=10, previous_val_loss=previous_val_loss)

In [100]:
previous_val_loss.append(history.history["val_loss"][-1])
performance_summary(model, X_test, y_test )

42.11% : Share of forecasts within 5% absolute error

1091   : Mean absolute error 

9.33% : Mean absolute percentage error



## Callbacks stop

In [66]:
tf.keras.backend.clear_session()
tf.random.set_seed(60)

model=keras.models.Sequential([
    
    keras.layers.Dense(1024, input_dim = X_train.shape[1]), 
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.4),
    
    keras.layers.Dense(512),  
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.4),

    keras.layers.Dense(512),  
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    
    keras.layers.Dense(units=256), 
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.2),
    
    keras.layers.Dense(units=256), 
    keras.layers.LeakyReLU(),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.1),

    
    keras.layers.Dense(units=128),
    keras.layers.LeakyReLU(), 
    keras.layers.Dense(units=1, activation="linear"),



],name="Callbacks",)


In [67]:
checkpoint_name = 'Weights\Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [68]:
optimizer = keras.optimizers.Adam(lr=0.001, decay=1e-4)



model.compile(optimizer=optimizer, warm_start=False, 
            loss='mean_absolute_error')


history = model.fit(X_train, y_train,
                    epochs=500, batch_size=1024,
                    validation_data=(X_test, y_test), 
                    callbacks=callbacks_list, 
                    verbose=1)

Train on 19948 samples, validate on 4987 samples
Epoch 1/500
19456/19948 [============================>.] - ETA: 0s - loss: 11194.7606
Epoch 00001: val_loss improved from inf to 11105.48480, saving model to Weights\Weights-001--11105.48480.hdf5
19948/19948 [==============================] - 4s 183us/sample - loss: 11192.1923 - val_loss: 11105.4848
Epoch 2/500
17408/19948 [=========================>....] - ETA: 0s - loss: 11186.7066- ETA: 0s - loss: 112
Epoch 00002: val_loss improved from 11105.48480 to 11046.21975, saving model to Weights\Weights-002--11046.21975.hdf5
19948/19948 [==============================] - 1s 40us/sample - loss: 11178.5528 - val_loss: 11046.2197
Epoch 3/500
17408/19948 [=========================>....] - ETA: 0s - loss: 11144.768 - ETA: 0s - loss: 11141.2362
Epoch 00003: val_loss improved from 11046.21975 to 10986.67571, saving model to Weights\Weights-003--10986.67571.hdf5
19948/19948 [==============================] - 1s 35us/sample - loss: 11150.9024 - val_lo

18432/19948 [==========================>...] - ETA: 0s - loss: 1412.9166
Epoch 00028: val_loss did not improve from 1536.23979
19948/19948 [==============================] - 1s 27us/sample - loss: 1412.5230 - val_loss: 2082.8408
Epoch 29/500
19456/19948 [============================>.] - ETA: 0s - loss: 1397.3998
Epoch 00029: val_loss did not improve from 1536.23979
19948/19948 [==============================] - 0s 23us/sample - loss: 1401.1522 - val_loss: 1742.0741
Epoch 30/500
19456/19948 [============================>.] - ETA: 0s - loss: 1396.5089
Epoch 00030: val_loss did not improve from 1536.23979
19948/19948 [==============================] - 0s 22us/sample - loss: 1393.9749 - val_loss: 1569.2119
Epoch 31/500
17408/19948 [=========================>....] - ETA: 0s - loss: 1394.9883
Epoch 00031: val_loss did not improve from 1536.23979
19948/19948 [==============================] - 0s 19us/sample - loss: 1388.8545 - val_loss: 1592.5279
Epoch 32/500
16384/19948 [===================

19948/19948 [==============================] - 1s 28us/sample - loss: 1287.7338 - val_loss: 1193.4373
Epoch 58/500
18432/19948 [==========================>...] - ETA: 0s - loss: 1278.8764
Epoch 00058: val_loss did not improve from 1193.43730
19948/19948 [==============================] - 0s 23us/sample - loss: 1281.0132 - val_loss: 1202.3001
Epoch 59/500
17408/19948 [=========================>....] - ETA: 0s - loss: 1289.4134
Epoch 00059: val_loss did not improve from 1193.43730
19948/19948 [==============================] - 1s 26us/sample - loss: 1282.8448 - val_loss: 1198.1756
Epoch 60/500
19456/19948 [============================>.] - ETA: 0s - loss: 1275.9570
Epoch 00060: val_loss improved from 1193.43730 to 1184.75829, saving model to Weights\Weights-060--1184.75829.hdf5
19948/19948 [==============================] - 1s 26us/sample - loss: 1274.2041 - val_loss: 1184.7583
Epoch 61/500
16384/19948 [=======================>......] - ETA: 0s - loss: 1265.5445
Epoch 00061: val_loss did

18432/19948 [==========================>...] - ETA: 0s - loss: 1227.5145
Epoch 00090: val_loss did not improve from 1160.97947
19948/19948 [==============================] - 0s 25us/sample - loss: 1231.5712 - val_loss: 1172.5581
Epoch 91/500
19456/19948 [============================>.] - ETA: 0s - loss: 1232.0958
Epoch 00091: val_loss did not improve from 1160.97947
19948/19948 [==============================] - 0s 25us/sample - loss: 1231.3971 - val_loss: 1162.7642
Epoch 92/500
17408/19948 [=========================>....] - ETA: 0s - loss: 1233.7182
Epoch 00092: val_loss did not improve from 1160.97947
19948/19948 [==============================] - 0s 25us/sample - loss: 1227.4570 - val_loss: 1167.9029
Epoch 93/500
19456/19948 [============================>.] - ETA: 0s - loss: 1219.6169
Epoch 00093: val_loss improved from 1160.97947 to 1157.57813, saving model to Weights\Weights-093--1157.57813.hdf5
19948/19948 [==============================] - 1s 28us/sample - loss: 1218.0871 - val_

Epoch 122/500
18432/19948 [==========================>...] - ETA: 0s - loss: 1187.6805
Epoch 00122: val_loss did not improve from 1141.19912
19948/19948 [==============================] - 0s 22us/sample - loss: 1186.7115 - val_loss: 1158.8678
Epoch 123/500
19456/19948 [============================>.] - ETA: 0s - loss: 1182.4951
Epoch 00123: val_loss did not improve from 1141.19912
19948/19948 [==============================] - 0s 21us/sample - loss: 1184.9158 - val_loss: 1142.4527
Epoch 124/500
18432/19948 [==========================>...] - ETA: 0s - loss: 1196.3520
Epoch 00124: val_loss improved from 1141.19912 to 1134.58526, saving model to Weights\Weights-124--1134.58526.hdf5
19948/19948 [==============================] - 1s 30us/sample - loss: 1194.1411 - val_loss: 1134.5853
Epoch 125/500
19456/19948 [============================>.] - ETA: 0s - loss: 1191.2348
Epoch 00125: val_loss did not improve from 1134.58526
19948/19948 [==============================] - 0s 21us/sample - loss:

...

Epoch 351/500
19456/19948 [============================>.] - ETA: 0s - loss: 1081.0937
Epoch 00351: val_loss did not improve from 1088.49181
19948/19948 [==============================] - 0s 18us/sample - loss: 1082.6691 - val_loss: 1096.6645
Epoch 352/500
18432/19948 [==========================>...] - ETA: 0s - loss: 1078.9748
Epoch 00352: val_loss did not improve from 1088.49181
19948/19948 [==============================] - 0s 19us/sample - loss: 1083.2092 - val_loss: 1092.5334
Epoch 353/500
16384/19948 [=======================>......] - ETA: 0s - loss: 1073.4323 ETA: 0s - loss: 1060.
Epoch 00353: val_loss did not improve from 1088.49181
19948/19948 [==============================] - 0s 20us/sample - loss: 1074.8193 - val_loss: 1099.0905
Epoch 354/500
18432/19948 [==========================>...] - ETA: 0s - loss: 1077.6155
Epoch 00354: val_loss did not improve from 1088.49181
19948/19948 [==============================] - 0s 19us/sample - loss: 1078.0548 - val_loss: 1096.0650
Epoch 

Epoch 384/500
17408/19948 [=========================>....] - ETA: 0s - loss: 1058.3956
Epoch 00384: val_loss did not improve from 1085.36634
19948/19948 [==============================] - 0s 18us/sample - loss: 1064.5961 - val_loss: 1092.0893
Epoch 385/500
19456/19948 [============================>.] - ETA: 0s - loss: 1063.7391
Epoch 00385: val_loss did not improve from 1085.36634
19948/19948 [==============================] - 0s 23us/sample - loss: 1063.7072 - val_loss: 1092.3078
Epoch 386/500
19456/19948 [============================>.] - ETA: 0s - loss: 1067.3431
Epoch 00386: val_loss did not improve from 1085.36634
19948/19948 [==============================] - 0s 20us/sample - loss: 1066.4729 - val_loss: 1091.0055
Epoch 387/500
19456/19948 [============================>.] - ETA: 0s - loss: 1069.8429
Epoch 00387: val_loss did not improve from 1085.36634
19948/19948 [==============================] - 0s 21us/sample - loss: 1071.0150 - val_loss: 1089.3437
Epoch 388/500
19456/19948 [=

18432/19948 [==========================>...] - ETA: 0s - loss: 1065.0218
Epoch 00417: val_loss did not improve from 1081.53856
19948/19948 [==============================] - 0s 19us/sample - loss: 1061.4676 - val_loss: 1084.4056
Epoch 418/500
17408/19948 [=========================>....] - ETA: 0s - loss: 1057.7991
Epoch 00418: val_loss did not improve from 1081.53856
19948/19948 [==============================] - 0s 19us/sample - loss: 1061.2709 - val_loss: 1088.9151
Epoch 419/500
19456/19948 [============================>.] - ETA: 0s - loss: 1058.7571
Epoch 00419: val_loss did not improve from 1081.53856
19948/19948 [==============================] - 0s 18us/sample - loss: 1058.2811 - val_loss: 1083.4411
Epoch 420/500
17408/19948 [=========================>....] - ETA: 0s - loss: 1058.1549
Epoch 00420: val_loss did not improve from 1081.53856
19948/19948 [==============================] - 0s 18us/sample - loss: 1055.9658 - val_loss: 1084.8916
Epoch 421/500
18432/19948 [===============

Epoch 451/500
19456/19948 [============================>.] - ETA: 0s - loss: 1056.0070
Epoch 00451: val_loss did not improve from 1079.95806
19948/19948 [==============================] - 0s 23us/sample - loss: 1053.6884 - val_loss: 1080.9800
Epoch 452/500
17408/19948 [=========================>....] - ETA: 0s - loss: 1048.2109
Epoch 00452: val_loss did not improve from 1079.95806
19948/19948 [==============================] - 0s 20us/sample - loss: 1049.7288 - val_loss: 1085.2457
Epoch 453/500
17408/19948 [=========================>....] - ETA: 0s - loss: 1043.8546
Epoch 00453: val_loss did not improve from 1079.95806
19948/19948 [==============================] - 0s 21us/sample - loss: 1047.8850 - val_loss: 1089.9407
Epoch 454/500
19456/19948 [============================>.] - ETA: 0s - loss: 1053.4880
Epoch 00454: val_loss did not improve from 1079.95806
19948/19948 [==============================] - 0s 21us/sample - loss: 1052.0199 - val_loss: 1091.4343
Epoch 455/500
19456/19948 [=

18432/19948 [==========================>...] - ETA: 0s - loss: 1043.6081
Epoch 00484: val_loss did not improve from 1077.85938
19948/19948 [==============================] - 0s 20us/sample - loss: 1043.0843 - val_loss: 1081.0226
Epoch 485/500
19456/19948 [============================>.] - ETA: 0s - loss: 1044.7068
Epoch 00485: val_loss did not improve from 1077.85938
19948/19948 [==============================] - 0s 21us/sample - loss: 1044.4006 - val_loss: 1084.4479
Epoch 486/500
17408/19948 [=========================>....] - ETA: 0s - loss: 1045.4683
Epoch 00486: val_loss did not improve from 1077.85938
19948/19948 [==============================] - 0s 20us/sample - loss: 1045.0449 - val_loss: 1079.6908
Epoch 487/500
19456/19948 [============================>.] - ETA: 0s - loss: 1040.1904
Epoch 00487: val_loss did not improve from 1077.85938
19948/19948 [==============================] - 0s 21us/sample - loss: 1041.9531 - val_loss: 1083.7413
Epoch 488/500
17408/19948 [===============

In [70]:
DNN_plot_loss(history,starting_epoch=10, previous_val_loss=previous_val_loss)

In [71]:
import glob
import os

list_of_files = glob.glob('Weights/*') # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getmtime)

In [72]:
weights_file =  latest_file # choose the best checkpoint 
model.load_weights(weights_file) # load it
model.compile(loss='mean_absolute_error', optimizer='adam')

In [73]:
performance_summary(model, X_test, y_test )

42.37% : Share of forecasts within 5% absolute error

1078   : Mean absolute error 

9.20% : Mean absolute percentage error

